In [234]:
#import needed libraries and data sheet. 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from matplotlib import rc

#Data URL, CSV download is only available format: https://www.kaggle.com/austinreese/craigslist-carstrucks-data/download
df = pd.read_csv('Data/vehicles_CL.csv')

In [235]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7034441763,https://saltlakecity.craigslist.org/cto/d/salt...,salt lake city,https://saltlakecity.craigslist.org,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,...,4wd,compact,hatchback,black,https://images.craigslist.org/00G0G_fTLDWM5Xyv...,PRICE REDUCED! -Garage kept -Low Miles (63K)...,NaN,ut,40.7372,-111.858
1,7034440610,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00v0v_7Cu0buIofU...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
2,7034440588,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,46463,2015.0,gmc,sierra 1500,excellent,NaN,...,4wd,NaN,NaN,white,https://images.craigslist.org/01515_lPvJ9bfbdY...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
3,7034440546,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_6Rjfp3NS4O...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
4,7034406932,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,49999,2018.0,ford,f-450,NaN,NaN,...,4wd,NaN,pickup,white,https://images.craigslist.org/00W0W_8yIUwRBXXd...,2018 Ford F-350 F350 F 350 SD Lariat Crew Cab ...,NaN,ut,40.3744,-104.694


In [236]:
#Drop features which cannot contribute meaningfully to our model.
df=df.drop(['url', 'region_url','image_url','county','id','fuel','vin','drive','size','region','lat', 'long','title_status','paint_color','description'], axis=1)

In [237]:
#Check for NA values within the df
df.isnull().sum()

price                0
year              1527
manufacturer     22764
model             7989
condition       231934
cylinders       199683
odometer         92324
transmission      3719
type            141531
state                0
dtype: int64

In [239]:
#Due to the volume of data, I chose to drop any row/columnn which contained NA values. 
#This shows how much it reduced the df size
print('start ',df.shape)
df.dropna(how='any',axis=0,inplace=True)
df.dropna(how='any',axis=1,inplace=True)
print('end ',df.shape)

start  (166557, 10)
end  (166557, 10)


In [241]:
#As a matter of domain knowledge, for the purposes of my business, I only wanted to deal with cars which fell within 
#the price, age, and milage parameters found below. This also removes erroneous entries or unrealistic records. 
df=df[(df.price > 2000) & (df.price < 55000)]
df=df[(df.year > 2000) & (df.year < 2020)]
df=df[(df.odometer >= 0) & (df.odometer < 200000)]

print('now ',df.shape)

now  (129198, 10)


In [41]:
#To further reduce the scope, it is understood that the location of sale also effects vehicle price. Since I will be doing
#business within my region of residence, I reduced the dataset further to include only sales which are within the following
#states.
MyRegion = df[(df['state'] =='pa') | (df['state'] =='md') | (df['state'] =='va') | (df['state'] =='wv')]

C:\Users\cyeager\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#Vehicle year is converted to age and added to a new column.
MyRegion['age'] = 2020-MyRegion['year']

In [242]:
#Finally, as a final outlier control, I removed all remaining records which fell 2 standard deviations outside of the mean.
#some stack overflow idea assistance here: https://stackoverflow.com/questions/46245035/pandas-dataframe-remove-outliers

print('start ',MyRegion.shape)
MyRegion[np.abs(MyRegion.price-MyRegion.price.mean())<=(2*MyRegion.price.std())] 
MyRegion[~(np.abs(MyRegion.price-MyRegion.price.mean())>(2*MyRegion.price.std()))]

MyRegion[np.abs(MyRegion.odometer-MyRegion.odometer.mean())<=(2*MyRegion.odometer.std())] 
MyRegion[~(np.abs(MyRegion.odometer-MyRegion.odometer.mean())>(2*MyRegion.odometer.std()))]

MyRegion[np.abs(MyRegion.age-MyRegion.age.mean())<=(2*MyRegion.age.std())]
MyRegion[~(np.abs(MyRegion.age-MyRegion.age.mean())>(2*MyRegion.age.std()))]

print('finish ',MyRegion.shape)

start  (9484, 19)
finish  (9484, 19)


Recoding the categorical variables for use with linear regression

In [244]:
#Get all possible values. condition is the only varibale we have in which rank matters, 
#so an ordered list is created manually from this
MyRegion['condition'].unique().tolist()

['excellent', 'good', 'like new', 'fair', 'new', 'salvage']

In [245]:
OrderedList=['salvage','fair','good','excellent','like new','new']

In [246]:
#remap the categorical variables within the df using the above to recode. 
#Stack overflow assistance with solving this: https://stackoverflow.com/questions/34007308/linear-regression-analysis-with-string-categorical-features-variables
MyRegion['Condition'] = MyRegion['condition'].astype('category')
MyRegion['Condition'] = MyRegion['Condition'].cat.reorder_categories(OrderedList, ordered=True)
MyRegion['Condition'] = MyRegion['Condition'].cat.codes
print(MyRegion['Condition'])

0       3
1       2
2       4
3       3
4       2
       ..
9479    3
9480    2
9481    2
9482    1
9483    2
Name: Condition, Length: 9484, dtype: int8


In [247]:
#Remove other and exotic values for engine size
MyRegion = MyRegion[~(MyRegion['cylinders'] =='other') & ~(MyRegion['cylinders'] =='10 cylinders')& ~(MyRegion['cylinders'] =='12 cylinders')]
MyRegion['cylinders'].value_counts()


6 cylinders    3709
4 cylinders    3227
8 cylinders    2444
5 cylinders      93
3 cylinders      11
Name: cylinders, dtype: int64

In [248]:
#Since we're not saying one is better than the next and simply recoding a list of possible values is automatically generated
#regular expression search removes the string portion and the loop below rewrites with lists as ints
MyRegion=MyRegion.reset_index()
MyRegion['Cylinders']=MyRegion.cylinders.str.findall('\d+')

In [249]:
Engines=[]
for i in range(len(MyRegion)):
    Engines.append([number for number in MyRegion['Cylinders'][i] if len(number)==1][0])
MyRegion['Cylinders']=Engines

In [250]:
MyRegion['Cylinders']=MyRegion['Cylinders'].astype(int)

In [251]:
#Same process for manufacturer. also not saying one is inherently better than the next.
MyRegion['Manufacturer'] = MyRegion['manufacturer'].astype('category')
MyRegion['Manufacturer'] = MyRegion['Manufacturer'].cat.reorder_categories(MyRegion['manufacturer'].unique().tolist(), ordered=False)
MyRegion['Manufacturer'] = MyRegion['Manufacturer'].cat.codes


In [252]:
#Same process for transmission. also not saying one is inherently better than the next.

MyRegion['Transmission'] = MyRegion['transmission'].astype('category')
MyRegion['Transmission'] = MyRegion['Transmission'].cat.reorder_categories(MyRegion['transmission'].unique().tolist(), ordered=False)
MyRegion['Transmission'] = MyRegion['Transmission'].cat.codes


In [253]:
#Same process for model. also not saying one is inherently better than the next.

MyRegion['Model'] = MyRegion['model'].astype('category')
MyRegion['Model'] = MyRegion['Model'].cat.reorder_categories(MyRegion['model'].unique().tolist(), ordered=False)
MyRegion['Model'] = MyRegion['Model'].cat.codes


In [255]:
#Same process for vehicle type (car truck etc). also not saying one is inherently better than the next.

MyRegion['Type'] = MyRegion['type'].astype('category')
MyRegion['Type'] = MyRegion['Type'].cat.reorder_categories(MyRegion['type'].unique().tolist(), ordered=False)
MyRegion['Type'] = MyRegion['Type'].cat.codes


In [256]:
MyRegion

,level_0,index,price,year,manufacturer,model,condition,cylinders,odometer,transmission,type,state,age,Condition,Cylinders,Manufacturer,Transmission,Model,Type,bin
0,0,3468,7999,2012.0,subaru,legacy,excellent,4 cylinders,122120.0,automatic,sedan,va,8.0,3,4,0,0,0,0,120000-130000
1,1,3652,2800,2004.0,toyota,camry le,good,4 cylinders,191000.0,automatic,sedan,va,16.0,2,4,1,0,1,0,190000-200000
2,2,3654,9450,2012.0,volvo,s80,like new,6 cylinders,92000.0,automatic,sedan,va,8.0,4,6,2,0,2,0,90000-100000
3,3,3677,2100,2002.0,volkswagen,new beetle gls,excellent,4 cylinders,141840.0,automatic,hatchback,va,18.0,3,4,3,0,3,1,140000-150000
4,4,3678,2900,2010.0,hyundai,elantra gls wagon,good,4 cylinders,186650.0,automatic,wagon,va,10.0,2,4,4,0,4,2,180000-190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9479,9479,450852,9900,2014.0,nissan,murano sl,excellent,6 cylinders,130569.0,automatic,SUV,pa,6.0,3,6,5,0,1458,6,130000-140000
9480,9480,450853,2300,2004.0,chevrolet,malibu maxx,good,6 cylinders,158000.0,automatic,wagon,pa,16.0,2,6,9,0,2097,2,150000-160000
9481,9481,450859,3200,2007.0,ford,focus,good,4 cylinders,117000.0,manual,hatchback,pa,13.0,2,4,13,2,255,1,110000-120000
9482,9482,450860,2300,2006.0,volkswagen,passat,fair,4 cylinders,122141.0,automatic,sedan,pa,14.0,1,4,3,0,147,0,120000-130000


In [257]:
MyRegion.to_csv('Cleaned_Filtered_Data.csv')